In [1]:
import qiskit
from qiskit import IBMQ
import math

In [4]:
_bitCache = ''

In [5]:
def set_provider(token = ''):
    global provider
    if token != '':
        IBMQ.save_account(token, overwrite=True) #save account for future use
        IBMQ.load_account() #load account using stored credentials
        provider = IBMQ.get_provider(hub='ibm-q') #public IBM Quantum devices available to all IBM Quantum users
    else: 
        provider = qiskit.BasicAer #Python based simulator that doesn't need IBMQ Account for access

In [6]:
def set_backend(b = 'qasm_simulator'):
    global _backend
    global provider
    _backend = provider.get_backend(b) #Choose quantum device to run on
    _set_circuit(_backend.configuration().n_qubits) #Develop our circuit, knowing the size of the quantum device

In [7]:
def _set_circuit(n):
    global _circuit
    qr = qiskit.QuantumRegister(n) #Implement a quantum register of n qubits
    cr = qiskit.ClassicalRegister(n) #Implement a classical register of n cbits
    _circuit = qiskit.QuantumCircuit(qr, cr) #Include the quantum register and the classical register in the circuit
    # n = 2
    # |0> ---------
    # |0> ---------
    # ============= 
    # =============
    _circuit.h(qr) # Apply Hadamard gate to all the qubits in the quantum register
    # n = 2
    # |0> --H------
    # |0> --H------
    # =============
    # =============
    _circuit.measure(qr,cr) # Measure all the qubits in the quantum register and store them in the classical register
    # n = 2
    # |0> --H--M---
    # |0> --H-----M
    # =============    <--- classical register now stores the measurement result of each qubit
    # =============

In [8]:
def _get_bits_from_counts(counts):
    for k, v in counts.items():
        if v == 1:
            return k
    #return [k for k, v in counts.items() if v == 1]

In [9]:
def _request_bits(n):
    global _bitCache #our primary data type for feeding and taking purely random bits
    #
    # suppose n=5 and the number of qubits is 8. We can generate 0, 5, 10, 15 ... random bits. 
    # That is, 0, 1, 2, 3 ... jobs. We want 10 random bits  <--> 2 jobs.
    # 8/5 = 1 but ceil(8/5) = 2. Generally, we want to use the ceiling operation for this task.
    #
    iterations = math.ceil(n * 2/_circuit.width()) #the width of the quantum register is half that of the circuit
    for _ in range(iterations):
        # Create new job and run the quantum circuit
        job = qiskit.execute(_circuit, _backend, shots=1) #submit circuit to backend once
        #
        # counts is a dictionary where the key is a binary string and the value is the count
        #
        _bitCache += _get_bits_from_counts(job.result().get_counts()) #convert the job to circuit.width()/2 new bits in the bitCache

In [10]:
def get_bit_string(n):
    global _bitCache
    if len(_bitCache) < n:
        _request_bits(n-len(_bitCache)) #request as many bits as we are lacking
    bitString = _bitCache[0:n] #result is the first n bits of the bitCache
    _bitCache = _bitCache[n:] #delete the first n bits of the bitCache
    return bitString

In [11]:
#get a random int between 0 and 2^16 - 1
def get_random_int16():
    return int(get_bit_string(16),2) #convert base 2 number to int

In [13]:
set_provider()
set_backend()
get_random_int16() 

52728

In [ ]:
set_provider('your token here')
set_backend(b = 'ibmq_manila')
get_random_int16() 

In [24]:
#View providers with your IBMQ account
IBMQ.load_account()
IBMQ.providers()

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>]

In [22]:
#View backends with the current provider
set_provider('[your token here]')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_bogota') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_extended_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_m